In [6]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [7]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000, 
                         epochs = 15, ## reduzido por falta de tempo, valor original: 25
                         validation_data = test_set,
                         validation_steps = 2000)

Epoch 1/15
8000/8000 [==============================] - 2745s 343ms/step - loss: 0.3917 - acc: 0.8122 - val_loss: 0.5327 - val_acc: 0.7979
Epoch 2/15
8000/8000 [==============================] - 2729s 341ms/step - loss: 0.1584 - acc: 0.9379 - val_loss: 0.8998 - val_acc: 0.7931
Epoch 3/15
8000/8000 [==============================] - 2722s 340ms/step - loss: 0.0777 - acc: 0.9718 - val_loss: 1.0719 - val_acc: 0.8021
Epoch 4/15
8000/8000 [==============================] - 2707s 338ms/step - loss: 0.0515 - acc: 0.9816 - val_loss: 1.1371 - val_acc: 0.8071
Epoch 5/15
8000/8000 [==============================] - 2804s 351ms/step - loss: 0.0398 - acc: 0.9863 - val_loss: 1.3258 - val_acc: 0.7998
Epoch 6/15
8000/8000 [==============================] - 3036s 380ms/step - loss: 0.0332 - acc: 0.9886 - val_loss: 1.3368 - val_acc: 0.7998
Epoch 7/15
8000/8000 [==============================] - 2815s 352ms/step - loss: 0.0282 - acc: 0.9903 - val_loss: 1.3137 - val_acc: 0.8125
Epoch 8/15
8000/8000 [=====

In [13]:
loss, acc = classifier.evaluate_generator(generator=test_set, steps=2000)

In [14]:
print(loss)

1.6844544234177925


In [15]:
print(acc)

0.8065933719758065
